In [2]:
CELLS = [(q, r) for q in range(-3, +3 + 1) for r in range(-3, +3 + 1) if -q - r in range(-3, +3 + 1)]

In [1]:
import tensorflow as tf
import numpy as np
from keras import models, layers
from keras.models import load_model
import json
import pickle
import os
from os.path import join
import subprocess
import json
import VanGame.utils as utils
import scipy as sp
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

Using TensorFlow backend.


In [3]:
sequence = sorted([p for p in CELLS])
sequence

[(-3, 0),
 (-3, 1),
 (-3, 2),
 (-3, 3),
 (-2, -1),
 (-2, 0),
 (-2, 1),
 (-2, 2),
 (-2, 3),
 (-1, -2),
 (-1, -1),
 (-1, 0),
 (-1, 1),
 (-1, 2),
 (-1, 3),
 (0, -3),
 (0, -2),
 (0, -1),
 (0, 0),
 (0, 1),
 (0, 2),
 (0, 3),
 (1, -3),
 (1, -2),
 (1, -1),
 (1, 0),
 (1, 1),
 (1, 2),
 (2, -3),
 (2, -2),
 (2, -1),
 (2, 0),
 (2, 1),
 (3, -3),
 (3, -2),
 (3, -1),
 (3, 0)]

In [4]:
a = np.array([[1]*39])
b = [500]

In [121]:
def build_model():
    model = models.Sequential()
    model.add(layers.Dense(1,activation='relu', input_shape=(10,)))
    #model.add(layers.Dense(200, activation='relu'))
    #model.add(layers.Dense(100, activation='relu'))
    model.add(layers.Dense(1, activation='tanh'))
    model.compile(optimizer='adam', loss='mse')
    return model

In [122]:
model = build_model()

In [7]:
def bash_call(c:[str]) -> None:
    res = subprocess.check_output(c)
    ls = []
    for line in res.splitlines():
        line = line.decode('utf-8')
        ls.append(line)
    return ls

In [8]:
def join_list(l):
    res = ""
    for i in l:
        res += i + "/"
    
    res = res[:-1]
    return res

In [9]:
def load_logs():
    datapath = join(os.getcwd(),'rec/')
    files = bash_call(['ls', datapath])
    return files

In [54]:
def PG_log(filename, condition=None, value_check=False):
    
    if (condition != None) and (condition not in filename):
        return False, False
    
    #print(filename)
    states, color = utils.load_l('rec/'+filename)
    
    train = []
    value = []
    
    for s in states:
        # print(s)
        board = []
        # add board
        for cord in sequence:
            # does not include the board
            break
            board.append(s['board'][cord])
        # color 
        # board.append(color)
        # utility = [heuristic diff, my pieces difference, danger pieces]
        board.append(s['turns'])
        board += s['utility']
        train.append(board)
        value.append(s['rew'])
        
    if value_check:
        print(value)
        
    value.reverse()
    decay = 0.7
    curr_decay = 0.7
    for i in range(1, len(value)):
        value[i] += value[i-1] * curr_decay
        curr_decay *= decay
    
    value.reverse()
    
    if value_check:
        print(value)
    
    post_value = np.array(value)
    
    post_value = np.tanh(post_value)
    
    # normalize the value
    # post_value = (post_value - post_value.mean()) / post_value.std()
    
    if value_check:
        print("\n",post_value)
    
    return train, post_value.tolist()

In [48]:
t,v = PG_log('[2zzz]1556678775.0caoredjiba.txt', value_check=True)
#print(t)

[0, 0, 0, -0.046875, -0.0125, -0.046875, 0.015625, -0.03125, -0.0125, -0.046875, 0, 0, 0, -0.0125, -0.046875, 0, -0.0125, -0.0125, 0, 125, -0.0125, -0.0125, -0.7625]
[-8.17053697283964e-12, -2.0897470424639635e-08, -3.7414063496715235e-05, -0.04688933556187888, -0.01257627309116172, -0.04683889354620397, 0.015520944856793693, -0.03131085384973177, -0.012817917072652849, -0.04687500106968943, -1.1040358669264306e-07, -7.976395915306688e-06, -0.000403392911340738, -0.014280646278525388, -0.0441260747403668, 0.04768465138056287, 0.5790183558182498, 5.027823065374546, 29.98942741342624, 124.9039042625, -0.2801624999999999, -0.5462499999999999, -0.7625]

 [-8.17053697e-12 -2.08974704e-08 -3.74140635e-05 -4.68550020e-02
 -1.25756101e-02 -4.68046706e-02  1.55196986e-02 -3.13006258e-02
 -1.28172151e-02 -4.68406989e-02 -1.10403587e-07 -7.97639592e-06
 -4.03392889e-04 -1.42796756e-02 -4.40974576e-02  4.76485420e-02
  5.21951584e-01  9.99914118e-01  1.00000000e+00  1.00000000e+00
 -2.73055471e-01

In [96]:
!ls rec

In [13]:
def reload_model():
    del model  # deletes the existing model

    # returns a compiled model
    # identical to the previous one
    model = load_model('my_model.h5')

In [14]:
def save_model():
    model.save('VanGame/my_model.h5')  # creates a HDF5 file 'my_model.h5'
    with open('VanGame/trained_model', 'wb+') as f:
        pickle.dump(model.get_weights(), f)

In [91]:
def train(epochs, save=True):
    logfs = load_logs()
    train_set = []
    target_set = []
    
    for logf in logfs:
        train, value = PG_log(logf)
        if train:
            #print(train)
            #print(len(train), len(value))
            #print(train[-1])
            train_set += train
            # the target shouldnt be calculate like this, should use bootstrapping
            target_set += value

    model.fit(np.array(train_set), target_set, epochs=epochs, batch_size=1)
    
    if save:
        print("saved")
        save_model()

In [128]:
model.predict(np.array([[-1]*10]))

array([[0.07565213]], dtype=float32)

In [124]:
save_model()

In [125]:
del model

In [127]:
try:
    model = load_model('VanGame/my_model.h5')
except:
    print("none model found")
    model = build_model()

In [84]:
def full_train_cycle(n, epochs=2):
    for i in range(n):
        !python3 -m referee VanGame HardCode HardCode -l logtry -v 0
    if (len(bash_call(['ls', 'rec']))) > 0:
        train(epochs=epochs)
        !rm -v ./rec/*

In [131]:
for i in range(1):
    print("********** starting round", i, "**********")
    full_train_cycle(1, epochs=2)

********** starting round 0 **********
* NOTE: unable to measure memory usage on this platform (try dimefox)
[1, -1, 0, 0, 0, 0, 0, 27, 28, 28]
[1, -1, 0, 0, 0, 0, 0, 27, 28, 28]
[1, -1, 0, 0, 0, 0, 0, 27, 28, 28]
[1, -1, 0, 0, 0, 0, 0, 27, 28, 28]
[1, -1, 0, 0, 0, 0, 0, 27, 28, 28]
[1, -1, 0, 0, 0, 0, 0, 27, 28, 28]
[1, -1, 0, 0, 0, 0, 0, 27, 28, 28]
[1, -1, 0, 0, 0, 0, 0, 27, 28, 28]
[2, 0, 0, 0, 0, 0, 0, 27, 27, 27]
[2, -2, 0, 0, 0, 0, 0, 25, 27, 27]
[2, -1, 0, 0, 0, 0, 0, 26, 27, 27]
[2, -1, 0, 0, 0, 0, 0, 26, 27, 27]
[2, -1, 0, 0, 0, 0, 0, 26, 27, 27]
[2, 0, 0, 0, 0, 0, 0, 27, 27, 27]
[2, -1, 0, 0, 0, 0, 0, 26, 27, 27]
[2, -1, 0, 0, 0, 0, 0, 26, 27, 27]
[2, 0, 0, 0, 0, 0, 0, 27, 27, 27]
[2, 0, 0, 0, 0, 0, 0, 27, 27, 27]
[2, -1, 0, 0, 0, 0, 0, 26, 27, 27]
[2, -1, 0, 0, 0, 0, 0, 26, 27, 27]
[2, 0, 0, 0, 0, 0, 0, 27, 27, 27]
[2, 1, 0, 0, 0, 0, 0, 28, 27, 27]
[3, 0, 0, 0, 0, 0, 0, 26, 25, 26]
[3, -1, 0, 0, 0, 0, 0, 25, 25, 26]
[3, -1, 0, 0, 0, 0, 0, 25, 25, 26]
[3, -1, 0, 1, 0, 0, 0, 

In [ ]:
[13, -1, 0, 0, 0, 1, 1, 31, 7, 13]

In [132]:
model.get_weights()

[array([[ 0.36815402],
        [-0.43592423],
        [ 0.6035008 ],
        [ 0.13034844],
        [-0.61405754],
        [-0.5820506 ],
        [-0.49549642],
        [ 0.19885297],
        [ 0.13539074],
        [ 0.16889122]], dtype=float32),
 array([-0.0450981], dtype=float32),
 array([[0.29974115]], dtype=float32),
 array([-0.04619823], dtype=float32)]

In [ ]:
for i in range(5000):
    print("********** starting round", i, "**********")
    full_train_cycle(1, epochs=2)

# CNN attamp

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten
#create model
md = Sequential()
#add model layers
md.add(Conv2D(1, kernel_size=3, activation='relu', input_shape=(28,28,1), use_bias=False))
#md.add(layers.MaxPooling2D(2,2))
#md.add(Conv2D(64, kernel_size=3, activation='relu', use_bias=False))
#md.add(layers.MaxPooling2D(2,2))
#md.add(Conv2D(64, kernel_size=3, activation='relu', use_bias=False))
md.add(Flatten())
md.compile(optimizer='adam', loss='mse')

In [ ]:
for i in md.get_weights():
    print(i.shape)

In [ ]:
md.summary()

In [ ]:
md.predict(np.array([[[[1]*28]*28]]).reshape(1,28,28,1))

In [ ]:
asdf = md.get_weights()[0].reshape(3,3)

In [ ]:
wds = md.get_weights()

In [ ]:
convp = wds[0]

In [ ]:
convp.shape

In [ ]:
c = convp.reshape(1, 3, 3, 1)

In [ ]:
d = c[0].reshape(3,3)

In [ ]:
import scipy.signal as signal

In [ ]:
a = np.array([[[1]*28]*28]).reshape(28,28)

In [ ]:
e = signal.convolve2d(a, asdf, mode='valid')

In [ ]:
e

In [ ]:
signal.convolve(e, )